In [70]:
import pandas as pd
import json

In [71]:
folder = '.\\input_files\\'

file = '341567805_uh-cleveland-medical-center_standardcharges.json'

In [72]:
file_path = folder + file
with open(file_path,'r') as f:
    jf = json.load(f)
jf = jf[0]

In [73]:
df = pd.DataFrame(jf['item'])

In [74]:
df = df.assign(Associated_Codes=df['Associated_Codes'].str.split(',')).explode('Associated_Codes')

In [75]:
df.rename(columns={
    'payer': 'payer_name',
    'Associated_Codes': 'code',
    'iobSelection': 'setting',
    'Payer_Allowed_Amount': 'standard_charge',
}, inplace=True)

In [76]:
df_payer = df.copy()
df_payer = df_payer[['payer_name', 'description', 'code', 'standard_charge', 'setting']]
df_payer['payer_category'] = 'payer'

In [77]:
df_rates = df.copy()
df_rates = df_rates[['description', 'code', 'setting', 'Avg_Gross_Charge', 'Cash_Discount', 'Deidentified_Min_Allowed', 'DeIdentified_Max_Allowed']]

In [78]:
cols = df_rates.columns.tolist()
id_vars = cols[:3]
value_vars = cols[3:]

df_rates = pd.melt(df_rates, id_vars=id_vars, value_vars=value_vars, var_name='payer_name', value_name='standard_charge')

In [79]:
mapping = {
    'Avg_Gross_Charge': 'gross',
    'Cash_Discount': 'cash',
    'Deidentified_Min_Allowed': 'min',
    'DeIdentified_Max_Allowed':'max',
    }

df_rates['payer_category'] = df_rates['payer_name'].map(mapping)

In [80]:
df = pd.concat([df_payer, df_rates])

In [81]:
df.reset_index(drop=True, inplace=True)

In [82]:
df['setting'] = df['setting'].str.lower()

In [83]:
df.loc[df['code'].str.match(r'^[A-Z][0-9]{4}|[0-9]{5}|[0-9]{4}[A-Z]$'), 'hcpcs_cpt'] = df['code']
df.loc[df['code'].str.match(r'[0-9]{3}'), 'ms_drg'] = df['code']
df.loc[df['code'].str.len() == 4, 'rev_code'] = df['code']

In [84]:
ccns = {'340714535': '360002',
 '340827442': '360041',
 '461382538': '360078',
 '341425870': '360098',
 '341260978': '360123',
 '341567805': '363302',
 '340714612': '360145',
 '340816492': '360192',
 '264827222': '360359',
 '340714461': '361307',
 '340714550': '361308',
 '371848577': '360367'}

ein = file.split('_')[0]
id = ccns[ein]

df['hospital_id'] = id

In [85]:
df.to_csv('.\\output_files\\' + ein + '_' id + '.csv', index=False)

,payer_name,description,code,standard_charge,setting,payer_category,hcpcs_cpt,ms_drg,rev_code,hospital_id
0,Advantage By Buckeye Medicare Advantage,Administration Of Vaccine,90460,57.1900,outpatient,payer,90460,90460,NaN,363302
1,Advantage By Buckeye Medicare Advantage,Administration Of Vaccine,90473,57.1900,outpatient,payer,90473,90473,NaN,363302
2,Advantage By Buckeye Medicare Advantage,Administration Of Vaccine,G0010,57.1900,outpatient,payer,G0010,NaN,NaN,363302
3,Advantage By Buckeye Medicare Advantage,Administration Of Vaccine,G0009,57.1900,outpatient,payer,G0009,NaN,NaN,363302
4,Advantage By Buckeye Medicare Advantage,Administration Of Vaccine,G0008,57.1900,outpatient,payer,G0008,NaN,NaN,363302
...,...,...,...,...,...,...,...,...,...,...
101960,DeIdentified_Max_Allowed,X-Ray Of Toes,73660,70.9800,outpatient,max,73660,73660,NaN,363302
101961,DeIdentified_Max_Allowed,X-Ray Of Upper Arm,73060,75.0500,outpatient,max,73060,73060,NaN,363302
101962,DeIdentified_Max_Allowed,X-Ray Of Upper Leg,73551,67.1800,outpatient,max,73551,73551,NaN,363302
101963,DeIdentified_Max_Allowed,X-Ray Of Upper Leg,73552,67.1800,outpatient,max,73552,73552,NaN,363302
